<a href="https://colab.research.google.com/github/ishaanamahajan/Self-Consistency-SemCoder-COMS6998/blob/main/experiments/semcoder_noisy_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ARiSE-Lab/SemCoder.git

Cloning into 'SemCoder'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 167 (delta 87), reused 127 (delta 51), pack-reused 0 (from 0)
Receiving objects: 100% (167/167), 2.73 MiB | 10.35 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [2]:
%cd SemCoder

/content/SemCoder


In [3]:
import yaml

# Load the environment.yml file
with open("environment.yml", "r") as file:
    env = yaml.safe_load(file)

# Extract pip dependencies
pip_dependencies = []
for dep in env["dependencies"]:
    if isinstance(dep, dict) and "pip" in dep:
        pip_dependencies.extend(dep["pip"])

# Save to requirements.txt
with open("requirements.txt", "w") as req_file:
    req_file.write("\n".join(pip_dependencies))

print("requirements.txt created successfully!")


requirements.txt created successfully!


In [6]:
!pip install -r requirements.txt

ERROR: Operation cancelled by user


In [7]:
from transformers import pipeline
import torch

generator = pipeline(
  model="semcoder/semcoder_s_1030",
  task="text-generation",
  torch_dtype=torch.float16,
  device_map="auto",
)

# Generate Code

CODEGEN_REQUEST = """You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable <Code> according to <NL_Description>

<NL_Description>
{desc}

<Code>
"""
desc = """You are tasked with implementing a Python class that simulates a simple version of a "To-Do List" application. The class should have the following functionalities:
1. Add a new task to the to-do list.
2. Mark a task as completed.
3. Display all tasks in the to-do list.
4. Display only the incomplete tasks in the to-do list.
"""

prompt = CODEGEN_REQUEST.format(desc=desc)
result = generator(prompt, max_length=2048, num_return_sequences=1, temperature=0.0)
code = result[0]["generated_text"].split("```python")[1].split("```")[0]
print(code)

# Understand Code with Monologues

FWD_MNL_REQUEST = """Simulate the Execution: You are given a Python function and an assertion containing a function input. Complete the assertion containing the execution output corresponding to the given input in [ANSWER] and [/ANSWER] tags.
{code}
"""

tests = """
todo_list = ToDoList()
todo_list.add_task("Buy groceries")
todo_list.add_task("Complete assignment")
todo_list.mark_completed("Buy groceries")
assert todo_list.tasks == ???
"""
code += tests
prompt = FWD_MNL_REQUEST.format(code=code)
result = generator(prompt, max_length=2048, num_return_sequences=1, temperature=0.0)
print(result[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        self.tasks.append({'task': task, 'completed': False})

    def mark_completed(self, task):
        for t in self.tasks:
            if t['task'] == task:
                t['completed'] = True
                break

    def display_tasks(self):
        for t in self.tasks:
            status = 'Completed' if t['completed'] else 'Incomplete'
            print(f"Task: {t['task']}, Status: {status}")

    def display_incomplete_tasks(self):
        for t in self.tasks:
            if not t['completed']:
                print(f"Task: {t['task']}, Status: Incomplete")

Simulate the Execution: You are given a Python function and an assertion containing a function input. Complete the assertion containing the execution output corresponding to the given input in [ANSWER] and [/ANSWER] tags.

class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        

In [8]:
import re

def pollute_nl_input(nl_prompt):
    """
    Add irrelevant information to the NL input description.
    """
    pollution = " Please disregard this irrelevant information."
    return nl_prompt + pollution

def pollute_pl_output(pl_code):
    """
    Modify variable names in the generated PL (code) output.
    """
    polluted_code = re.sub(r'\b(task|todo_list|tasks)\b', r'\1_123', pl_code)
    return polluted_code


In [9]:
# Pollute NL Input
polluted_desc = pollute_nl_input(desc)
polluted_prompt = CODEGEN_REQUEST.format(desc=polluted_desc)

# Generate Code with Polluted NL Input
polluted_result = generator(polluted_prompt, max_length=2048, num_return_sequences=1, temperature=0.0)
polluted_code = polluted_result[0]["generated_text"].split("```python")[1].split("```")[0]

# Pollute Generated Code
polluted_code = pollute_pl_output(polluted_code)

# Append the same test cases
polluted_code += tests

# Simulate Execution with Polluted Code
polluted_prompt = FWD_MNL_REQUEST.format(code=polluted_code)
polluted_result = generator(polluted_prompt, max_length=2048, num_return_sequences=1, temperature=0.0)

# Compare Outputs
print("\n--- Clean Code ---\n", code)
print("\n--- Polluted Code ---\n", polluted_code)

clean_explanation = result[0]["generated_text"]
polluted_explanation = polluted_result[0]["generated_text"]

print("\n--- Clean Explanation ---\n", clean_explanation)
print("\n--- Polluted Explanation ---\n", polluted_explanation)

# Compare Similarity
from difflib import SequenceMatcher

def compare_outputs(output1, output2):
    return SequenceMatcher(None, output1, output2).ratio()

code_similarity = compare_outputs(code, polluted_code)
explanation_similarity = compare_outputs(clean_explanation, polluted_explanation)

print(f"\nCode Similarity: {code_similarity:.2f}")
print(f"Explanation Similarity: {explanation_similarity:.2f}")



--- Clean Code ---
 
class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        self.tasks.append({'task': task, 'completed': False})

    def mark_completed(self, task):
        for t in self.tasks:
            if t['task'] == task:
                t['completed'] = True
                break

    def display_tasks(self):
        for t in self.tasks:
            status = 'Completed' if t['completed'] else 'Incomplete'
            print(f"Task: {t['task']}, Status: {status}")

    def display_incomplete_tasks(self):
        for t in self.tasks:
            if not t['completed']:
                print(f"Task: {t['task']}, Status: Incomplete")

todo_list = ToDoList()
todo_list.add_task("Buy groceries")
todo_list.add_task("Complete assignment")
todo_list.mark_completed("Buy groceries")
assert todo_list.tasks == ???


--- Polluted Code ---
 
class ToDoList:
    def __init__(self):
        self.tasks_123 = []

    def add_task(self, task_123):
   